# End to End classification audio data
1. Problem
Identify the class of sound each audio contains

2. Data
Our data come from kaggle website

3. Evaluation

 Mean Average Precision

 4. feature


In [ ]:
# Make clean structure under Audio_practice
#!mkdir -p "/content/drive/MyDrive/Audio_practice/FSDKaggle2018/train"
#!mkdir -p "/content/drive/MyDrive/Audio_practice/FSDKaggle2018/test"
#!mkdir -p "/content/drive/MyDrive/Audio_practice/FSDKaggle2018/meta"
#!mkdir -p "/content/drive/MyDrive/Audio_practice/FSDKaggle2018/doc"

# Unzip each file into its place
#!unzip -q "/content/drive/MyDrive/Audio_practice/FSDKaggle2018.audio_train.zip" -d "/content/drive/MyDrive/Audio_practice/FSDKaggle2018/train"
#!unzip -q "/content/drive/MyDrive/Audio_practice/FSDKaggle2018.audio_test.zip" -d "/content/drive/MyDrive/Audio_practice/FSDKaggle2018/test"
#!unzip -q "/content/drive/MyDrive/Audio_practice/FSDKaggle2018.meta.zip" -d "/content/drive/MyDrive/Audio_practice/FSDKaggle2018/meta"
#!unzip -q "/content/drive/MyDrive/Audio_practice/FSDKaggle2018.doc.zip" -d "/content/drive/MyDrive/Audio_practice/FSDKaggle2018/doc"

# Get the workplace ready

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
#!ls -lh "/content/drive/MyDrive/Audio_practice/FSDKaggle2018/meta"

In [ ]:
#!ls -lh "/content/drive/MyDrive/Audio_practice/FSDKaggle2018/meta/FSDKaggle2018.meta"

In [ ]:
meta = pd.read_csv("/content/drive/MyDrive/Audio_practice/FSDKaggle2018/meta/FSDKaggle2018.meta/train_post_competition.csv")
print(meta.head())

          fname         label  manually_verified  freesound_id  \
0  00044347.wav        Hi-hat                  0         28739   
1  001ca53d.wav     Saxophone                  1        358827   
2  002d256b.wav       Trumpet                  0         10897   
3  0033e230.wav  Glockenspiel                  1        325017   
4  00353774.wav         Cello                  1        195688   

              license  
0         Attribution  
1         Attribution  
2  Creative Commons 0  
3         Attribution  
4         Attribution  


In [ ]:
print("Number of classes:", meta['label'].nunique())
print("Classes:", sorted(meta['label'].unique())[:10])  # show first 10


Number of classes: 41
Classes: ['Acoustic_guitar', 'Applause', 'Bark', 'Bass_drum', 'Burping_or_eructation', 'Bus', 'Cello', 'Chime', 'Clarinet', 'Computer_keyboard']


In [ ]:
# Add full path to each audio file
#meta['path'] = "/content/drive/MyDrive/Audio_practice/FSDKaggle2018/train/FSDKaggle2018.audio_train" + meta['fname']

# Check the first few rows
#print(meta[['fname', 'label', 'path']].head())

In [ ]:
import os
import pandas as pd
import tensorflow as tf

# point to the folder that actually contains the wav files (no trailing slash needed)
TRAIN_DIR = "/content/drive/MyDrive/Audio_practice/FSDKaggle2018/train/FSDKaggle2018.audio_train"

# clean the filename column and join with os.path.join (adds the slash correctly)
meta["fname"] = meta["fname"].astype(str).str.strip()
meta["path"]  = meta["fname"].apply(lambda f: os.path.join(TRAIN_DIR, f))

# sanity checks
print("Example path:", meta["path"].iloc[0])
print("Exists?     ", os.path.exists(meta["path"].iloc[0]))

# find any bad paths
bad = meta.loc[~meta["path"].apply(os.path.exists), ["fname","path"]]
print("Missing files:", len(bad))
print(bad.head())
assert len(bad) == 0, "Some paths are wrong—see printed rows above."

Example path: /content/drive/MyDrive/Audio_practice/FSDKaggle2018/train/FSDKaggle2018.audio_train/00044347.wav
Exists?      True
Missing files: 0
Empty DataFrame
Columns: [fname, path]
Index: []


In [ ]:
# meta["label"].value_counts()

,count
label,
Hi-hat,300
Saxophone,300
Trumpet,300
Cello,300
Clarinet,300
Tearing,300
Fart,300
Laughter,300
Snare_drum,300


In [ ]:
# meta["fname"].value_counts()

,count
fname,
fff81f55.wav,1
ffbba18f.wav,1
ffb4c87e.wav,1
ffa61f00.wav,1
ffa4a506.wav,1
...,...
003b91e8.wav,1
00353774.wav,1
0033e230.wav,1


In [ ]:
# let's convert our label into numeric

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
meta["label_idx"] = le.fit_transform(meta["label"])
print(meta[["fname", "label", "label_idx"]])
print("classes:", list(le.classes_))


             fname         label  label_idx
0     00044347.wav        Hi-hat         23
1     001ca53d.wav     Saxophone         30
2     002d256b.wav       Trumpet         38
3     0033e230.wav  Glockenspiel         19
4     00353774.wav         Cello          6
...            ...           ...        ...
9468  ffec59fb.wav     Fireworks         17
9469  fff37590.wav        Hi-hat         23
9470  fff44ac6.wav      Laughter         26
9471  fff6a13d.wav         Chime          7
9472  fff81f55.wav         Cough         10

[9473 rows x 3 columns]
classws: ['Acoustic_guitar', 'Applause', 'Bark', 'Bass_drum', 'Burping_or_eructation', 'Bus', 'Cello', 'Chime', 'Clarinet', 'Computer_keyboard', 'Cough', 'Cowbell', 'Double_bass', 'Drawer_open_or_close', 'Electric_piano', 'Fart', 'Finger_snapping', 'Fireworks', 'Flute', 'Glockenspiel', 'Gong', 'Gunshot_or_gunfire', 'Harmonica', 'Hi-hat', 'Keys_jangling', 'Knock', 'Laughter', 'Meow', 'Microwave_oven', 'Oboe', 'Saxophone', 'Scissors', 'Shatter',

In [ ]:
# set x and y variable
X = meta["path"]
y = meta["label_idx"]
len(X), len(y)

(9473, 9473)

Create our own train and valid section

In [ ]:
# let spli oufr data into train and valid
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

Preprocess our data

In [ ]:
import tensorflow as tf

def preprocess_audio(file_path):
  # 1. Load audio
  audio = tf.io.read_file(file_path)
  # 2. Decode audio
  audio , sample_rate = tf.audio.decode_wav(audio)
  # remove the channel dimention
  audio = tf.squeeze(audio, axis=1)
  # Normalize (-1 to 1)
  denom = tf.maximum(tf.reduce_max(tf.abs(audio)), 1e-7)
  audio = audio / denom
  # Convert to spectrogram
  spectrogram = tf.signal.stft(audio, frame_length=256, frame_step=128)
  spectrogram = tf.abs(spectrogram)
  # Convert to mel-spectogram
  mel_filter = tf.signal.linear_to_mel_weight_matrix(
      num_mel_bins=64,
      num_spectrogram_bins=spectrogram.shape[-1],
      sample_rate=sample_rate,
      lower_edge_hertz=80,
      upper_edge_hertz=7600
  )
  mel_spectrogram = tf.tensordot(spectrogram, mel_filter, 1)
  # 5. Add channel dimension(like image)
  mel_spectrogram = tf.expand_dims(mel_spectrogram, -1)
  return mel_spectrogram

In [ ]:
#import tensorflow as tf

#def mel_to_rgb224(mel):                 # mel: [T, 64, 1]
  #  x = tf.image.resize(mel, [224, 224])        # -> [224,224,1]
    # scale per example to [0,1] (good for TF-Hub MobileNet feature_vector)
    #mn = tf.reduce_min(x)
   # x = (x - mn) / (mx - mn + 1e-6)
  #  x = tf.repeat(x, repeats=3, axis=-1)        # -> [224,224,3]
  #  return x

In [ ]:
#def get_image_and_label(path, label):
   # mel = preprocess_audio(path)                 # your function -> [T,64,1]
  #  img = mel_to_rgb224(mel)                     # -> [224,224,3]
   # return img, label


In [ ]:

#def get_image_only(path):                        # for test set (paths only)
    #mel = preprocess_audio(path)
    #return mel_to_rgb224(mel)

In [ ]:
# make labels 1-D ints inside your dataset map
def map_to_img224(path, label):
    mel = preprocess_audio(path)
    x = tf.image.resize(mel, [224, 224])
    x = (x - tf.reduce_min(x)) / (tf.reduce_max(x) - tf.reduce_min(x) + 1e-6)
    x = tf.repeat(x, repeats=3, axis=-1)  # -> [224,224,3]
    y = tf.cast(tf.reshape(label, []), tf.int32)  # ( ) not (1,)
    return x, y

# Turn data into batcbh

In [ ]:
# create a batch function
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 32
X = tf.constant(X)
y = tf.constant(y)
def create_audio_batches(X,
                         y=None,
                         batch_size = BATCH_SIZE,
                         valid_data = False,
                         test_data = False,
                         padded = True,
                         n_mels = 64
                         ):
  """ This function create batch data of audio and label
  """
  if test_data:
    print("Creating test data batches...")
    data = tf.data.Dataset.from_tensor_slices(X)
    data = data.map(preprocess_audio, num_parallel_calls=AUTOTUNE)

    data_batch = data.batch(batch_size)
    return data_batch
  elif valid_data:
    print("Creating validation data batches...")
    data = tf.data.Dataset.from_tensor_slices((X, y))
    data = data.map(map_to_img224, num_parallel_calls=AUTOTUNE)

    data_batch = data.batch(batch_size)
    return data_batch
  else:
    print("Creating training data batches...")
    data = tf.data.Dataset.from_tensor_slices((X, y))
    data = data.shuffle(buffer_size=len(X))
    data = data.map(map_to_img224, num_parallel_calls=AUTOTUNE)

    data_batch = data.batch(batch_size)
    return data_batch



In [ ]:
# Creating taring and validation batch
train_data = create_audio_batches(X_train, y_train)
val_data = create_audio_batches(X_val, y_val, valid_data=True)

Creating training data batches...
Creating validation data batches...


In [ ]:
num_class = meta["label"].unique()
num_class[:10]

array(['Hi-hat', 'Saxophone', 'Trumpet', 'Glockenspiel', 'Cello', 'Knock',
       'Gunshot_or_gunfire', 'Clarinet', 'Computer_keyboard',
       'Keys_jangling'], dtype=object)

In [ ]:
len(num_class)

41

Build the model

In [ ]:
# set up model input and output
# Set imput shape to the model
IMG_SIZE = 224
INPUT_SHAPE = [ IMG_SIZE, IMG_SIZE, 3] # batch, height, width, color channel

#Ste up output shape of our model
OUTPUT_SHAPE = len(num_class)

# Setup model URL
MODEL_URL = "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/feature_vector/5"


In [ ]:
from tf_keras import layers, models, losses, optimizers, metrics
import tensorflow_hub as hub

# create a functio which build a Keras model
def create_model(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, model_url=MODEL_URL):
  print("Building model with:", MODEL_URL)
  # steup the model layers
  model = models.Sequential([
    hub.KerasLayer(MODEL_URL, input_shape=(224,224,3), trainable=False),
    layers.Dense(OUTPUT_SHAPE, activation="softmax"),
   ])
  # compile the model
  model.compile(loss=losses.SparseCategoricalCrossentropy(),
              optimizer=optimizers.Adam(),
              metrics=[
                  metrics.SparseCategoricalAccuracy(name="accuracy"),

                  metrics.SparseTopKCategoricalAccuracy(k=3, name="top-3-accuracy")
                  ]
             )


              #metrics=["accuracy"]
  return model

In [ ]:
model = create_model()
model.summary()

Building model with: https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/feature_vector/5
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1664)              3766048   
                                                                 
 dense (Dense)               (None, 41)                68265     
                                                                 
Total params: 3834313 (14.63 MB)
Trainable params: 68265 (266.66 KB)
Non-trainable params: 3766048 (14.37 MB)
_________________________________________________________________


# creation of call back


In [ ]:
# call back
%load_ext tensorboard

In [ ]:
import datetime
from tf_keras.callbacks import TensorBoard   # <-- import the class

# create a function to build tensorboard callback
def create_tensorboard_callback():
  # create a log directory for storing Tensorboard logs
  logdir = os.path.join("drive/MyDrive/Audio_practice/FSDKaggle2018/my_model",
                        # make it so the logs get tracked whenever we run an experiment
                        datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  #return tf_keras.callbacks.TensorBoard(log_dir=logdir)
  return TensorBoard(log_dir=logdir)

In [ ]:
# Early stop function
from tf_keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor="val_accuracy",   # works if you compiled with metrics=["accuracy"]
    patience=3,
    mode="max",
    restore_best_weights=True,
    min_delta=1e-4            # ignore tiny fluctuations
)

# Training on the model

In [ ]:
# creation of Epoch
NUM_EPOCHS = 100#@param {type:"slider", min:10, max:100, step:10}

In [ ]:
# Build a function to train and return a trained model
def train_model():
  """
  Trains a given model return the trauined version.
  """
  # create a model
  model = create_model()

  # Create new Tensorboard session everytime we train a model
  tensorboard = create_tensorboard_callback()

  # fit ten model to the data passing it thhe callbacks we created
  model.fit(x=train_data,
            epochs=NUM_EPOCHS,
            validation_data=val_data,
            validation_freq=1,
            callbacks=[tensorboard, early_stopping])
  return model


In [ ]:
#model = train_model()
#model.save("/content/drive/MyDrive/Audio_practice/FSDKaggle2018/save/my_audio_model.keras")

In [ ]:
from tf_keras.models import load_model
model = load_model(
    "/content/drive/MyDrive/Audio_practice/FSDKaggle2018/save/my_audio_model.keras",
    custom_objects={"KerasLayer": hub.KerasLayer}
    )

In [ ]:
#model.evaluate(val_data)

In [ ]:
predictions = model.predict(val_data, verbose =1)
predictions

60/60 [==============================] - 300s 5s/step


array([[5.0046379e-05, 1.1007405e-06, 1.4985225e-04, ..., 4.0902663e-04,
        1.0060959e-06, 2.8905593e-02],
       [7.3420422e-05, 3.6970970e-08, 3.6216585e-03, ..., 2.5682725e-02,
        4.1531189e-04, 9.4741226e-06],
       [1.4524257e-03, 8.9489770e-01, 2.0901910e-03, ..., 1.0760581e-03,
        3.4827215e-04, 1.2946565e-03],
       ...,
       [2.5282650e-05, 4.4243635e-09, 1.6715400e-07, ..., 2.5116804e-05,
        2.6687073e-07, 8.2260050e-04],
       [2.8077504e-01, 1.9035445e-03, 8.3961198e-03, ..., 6.3956003e-03,
        3.5848296e-03, 1.1845762e-01],
       [7.1603702e-03, 3.1309539e-01, 3.2416400e-02, ..., 1.1275397e-01,
        1.2385923e-02, 2.1628193e-03]], dtype=float32)

In [ ]:
# Mean average precision
from sklearn.metrics import average_precision_score
Ap = average_precision_score(y_val, predictions)

In [ ]:
print("average precision:", Ap)

average precision: 0.6557386994855422


In [ ]:
MAp = np.mean(Ap)
print("Mean average precision:", MAp)

Mean average precision: 0.6557386994855422


In [ ]:
predictions.shape

(1895, 41)

In [ ]:
y_val.shape

(1895,)

In [ ]:
# First prediction
index = 0
print(predictions[index])
print(f"Max value (probability of prediction): {np.max(predictions[index])}")
print(f"Sum: {np.sum(predictions[index])}")
print(f"Max index: {np.argmax(predictions[index])}")
print(f"Predicted label: {num_class[np.argmax(predictions[index])]}")

[5.0046379e-05 1.1007405e-06 1.4985225e-04 2.5537165e-05 1.8452440e-04
 2.3336696e-02 7.7941127e-07 2.5970879e-01 1.4622830e-04 4.2199881e-06
 6.9853291e-04 3.6446012e-03 2.1048912e-09 1.0593293e-06 2.6877613e-05
 7.1911857e-02 7.8286271e-04 1.5211092e-01 3.8413945e-04 2.3435015e-02
 6.1330793e-05 4.5818007e-05 1.4030862e-06 1.0966798e-03 1.8132377e-02
 3.0426570e-04 8.4087448e-03 2.7799329e-03 1.6303295e-05 1.2182430e-07
 1.1968478e-07 8.6999964e-05 6.0378946e-03 1.3892779e-05 1.7831320e-01
 2.1773796e-01 9.7484648e-04 6.8867412e-05 4.0902663e-04 1.0060959e-06
 2.8905593e-02]
Max value (probability of prediction): 0.25970879197120667
Sum: 0.9999999403953552
Max index: 7
Predicted label: Clarinet


In [ ]:
num_class

array(['Hi-hat', 'Saxophone', 'Trumpet', 'Glockenspiel', 'Cello', 'Knock',
       'Gunshot_or_gunfire', 'Clarinet', 'Computer_keyboard',
       'Keys_jangling', 'Snare_drum', 'Writing', 'Laughter', 'Tearing',
       'Fart', 'Oboe', 'Flute', 'Cough', 'Telephone', 'Bark', 'Chime',
       'Bass_drum', 'Bus', 'Squeak', 'Scissors', 'Harmonica', 'Gong',
       'Microwave_oven', 'Burping_or_eructation', 'Double_bass',
       'Shatter', 'Fireworks', 'Tambourine', 'Cowbell', 'Electric_piano',
       'Meow', 'Drawer_open_or_close', 'Applause', 'Acoustic_guitar',
       'Violin_or_fiddle', 'Finger_snapping'], dtype=object)

In [ ]:
import numpy as np
import pandas as pd

# Convert predictions to class indices
y_pred_labels = np.argmax(predictions, axis=1)

# True labels are already 1D
y_true_labels = y_val

# Max probability per sample
max_prob = predictions.max(axis=1)

# Build DataFrame
df = pd.DataFrame({
    "y_true": y_true_labels,
    "y_pred": y_pred_labels,
    "correct": (y_true_labels == y_pred_labels),
    "confidence": max_prob
})

print(df.head(7))

      y_true  y_pred  correct  confidence
3593      17       7    False    0.259709
2273      37      18    False    0.352068
8395       1       1     True    0.894898
967       38       5    False    0.206310
8966      29      29     True    0.767611
9400      15      30    False    0.305595
6876      40      17    False    0.261145


In [ ]:
pL[0]

'C'

In [ ]:
import numpy as np
import pandas as pd

# Convert predictions to class indices
y_pred_labels = np.argmax(predictions, axis=1)

# True labels are already 1D
y_true_labels = y_val

# Max probability per sample
max_prob = predictions.max(axis=1)
class_names = np.array(num_class)
pred_names = class_names[y_pred_labels]
true_names = class_names[y_true_labels]

# Build DataFrame
df = pd.DataFrame({
    "y_true": y_true_labels,
    "y_pred": y_pred_labels,
    "correct": (y_true_labels == y_pred_labels),
    "confidence": max_prob,
    "true_name": true_names,
    "pred_name": pred_names
})

print(df.head(7))

      y_true  y_pred  correct  confidence        true_name    pred_name
3593      17       7    False    0.259709            Cough     Clarinet
2273      37      18    False    0.352068         Applause    Telephone
8395       1       1     True    0.894898        Saxophone    Saxophone
967       38       5    False    0.206310  Acoustic_guitar        Knock
8966      29      29     True    0.767611      Double_bass  Double_bass
9400      15      30    False    0.305595             Oboe      Shatter
6876      40      17    False    0.261145  Finger_snapping        Cough


In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, y_pred_labels)
print("Accuracy:", accuracy)

Accuracy: 0.6089709762532981
